# Things done these two weeks (2018-11-1)

## Re-implemented the processing of scores and features

In [1]:
from piano_reduction import tools as pr, features as ft, compute_features as cf
from piano_reduction.score_data import ScoreData
import imp
imp.reload(pr)
imp.reload(ft)
imp.reload(cf)

<module 'piano_reduction.compute_features' from '/Users/jason2yik/fyp/piano-reduction-main/piano_reduction/compute_features.py'>

In [222]:
data = ScoreData(pr.load_cosi(2))
data.df.head()

,measure,offset,part,ps,tie,x_train,color,y_train,duration,pitch,...,in_chord,lowest,occurrence,offset_value,onset_after_rest,pitch_distance,rhythm_variety,strong_beats,sustained_rhythm,vertical_doubling
3843,0.0,0,-1,43.0,None,0,NaN,1,<music21.duration.Duration 0.75>,G2,...,0,0,0,0.0,0,0.0,0,0,0,0
0,0.0,0,11,55.0,None,1,None,0,<music21.duration.Duration 0.75>,G3,...,0,1,1,0.0,0,5.0,1,1,1,1
3844,0.0,0,-1,59.0,None,0,NaN,1,<music21.duration.Duration 0.75>,B3,...,0,0,0,0.0,0,0.0,0,0,0,0
1,0.0,0,9,62.0,None,1,None,1,<music21.duration.Duration 0.75>,D4,...,0,0,1,0.0,0,2.0,1,1,1,1
3845,0.0,0.75,-1,43.0,None,0,NaN,1,<music21.duration.Duration 0.25>,G2,...,0,0,0,0.0,0,0.0,0,0,0,0


In [22]:
data.save('score_data/cosi_2.pkl')

In [23]:
data = data.load('score_data/cosi_2.pkl')

#### Split the data of the same score

In [14]:
training_data, testing_data = data.split(0.5)
training_data.df.shape, testing_data.df.shape

((2634, 25), (1733, 25))

## Evaluate the models using the last-half of the score as training data and the first-half as testing data

In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras import backend as K
from keras.callbacks import BaseLogger
from keras.callbacks import EarlyStopping
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

early_stop = EarlyStopping(monitor='val_acc', mode='max', min_delta=0, patience=100, verbose=0, restore_best_weights=True)

Using TensorFlow backend.


In [3]:
nn = Sequential()
nn.add(Dense(200, input_shape=[cf.feature_count()]))
nn.add(Activation('sigmoid'))
nn.add(Dense(1))
nn.add(Activation('sigmoid'))
nn.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', f1])
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               3200      
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 3,401
Trainable params: 3,401
Non-trainable params: 0
_________________________________________________________________


In [4]:
lstm = Sequential()
lstm.add(LSTM(200, input_shape=(None, cf.feature_count())))
lstm.add(Dropout(0.2))
lstm.add(Dense(1))
lstm.add(Activation('sigmoid'))
lstm.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', f1])
lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               172800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 173,001
Trainable params: 173,001
Non-trainable params: 0
_________________________________________________________________


In [5]:
data = ScoreData.load('score_data/cosi_2.pkl')
testing_data, training_data = data.split()

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

x_train, y_train = training_data.generate_data(length=-1)
x_test, y_test = testing_data.generate_data(length=-1)
lr = LogisticRegression().fit(x_train, y_train)
nb = MultinomialNB().fit(x_train, y_train)
nn.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop], verbose=0)

x_train, y_train = training_data.generate_data(length=10)
x_test, y_test = testing_data.generate_data(length=10)
lstm.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop], verbose=0)

In [6]:
nn.save('models/NN.h5')
lstm.save('models/LSTM.h5')

#### With some baseline algorithms

In [240]:
class KeepAll():
    @classmethod
    def predict(cls, x):
        return [1 for _ in range(x.shape[0])]
class KeepSome():
    @classmethod
    def predict(cls, x):
        from random import randint
        return [randint(0, 1) for _ in range(x.shape[0])]

#### Results using the same piece (Cosi)

In [241]:
from piano_reduction.score_data import ScoreData

In [267]:
data = ScoreData.load('score_data/cosi_2.pkl')
validation_data, _ = data.split(0.5)
from sklearn import metrics
print('%30s %11s %11s' % ('', 'Accuracy', 'F1 Score'))
models = [('Keep All', KeepAll, -1),
          ('Keep Some', KeepSome, -1),
          ('Logistic Regression', lr, -1), 
          ('Naive Bayes', nb, -1), 
          ('Dense Network', nn, -1), 
          ('LSTM with 1 timestep', lstm, 1), 
          ('LSTM with 10 timesteps', lstm, 10)]
for name, model, length in models:
    validation_data.get_y_pred(model, length, threshold=0.5)
    print('%30s %.9f %.9f' % (name, 
                              metrics.accuracy_score(validation_data.df['y_train'], validation_data.df['y_pred']), 
                              metrics.f1_score(validation_data.df['y_train'], validation_data.df['y_pred'])))

                                  Accuracy    F1 Score
                      Keep All 0.490129081 0.657834395
                     Keep Some 0.415337889 0.440000000
           Logistic Regression 0.524677297 0.650864473
                   Naive Bayes 0.441154138 0.561121049
                 Dense Network 0.540622627 0.659730034
          LSTM with 1 timestep 0.482156416 0.647909138
        LSTM with 10 timesteps 0.509870919 0.547176429


In [268]:
validation_data, _ = data.split(0.5)
for name, model, length in models:
    validation_data.get_y_pred(model, length)
    validation_data.df[name] = validation_data.df['y_pred']
validation_data.show_score(col=['x_train'] + [i[0] for i in models] + ['y_train'])

#### Results using another piece (Symphony No.5)

In [257]:
data = ScoreData.load('score_data/2.pkl')
validation_data, _ = data.split(0.5)
from sklearn import metrics
print('%30s %11s %11s' % ('', 'Accuracy', 'F1 Score'))
models = [('Keep All', KeepAll, -1),
          ('Keep Some', KeepSome, -1),
          ('Logistic Regression', lr, -1), 
          ('Naive Bayes', nb, -1), 
          ('Dense Network', nn, -1), 
          ('LSTM with 1 timestep', lstm, 1), 
          ('LSTM with 10 timesteps', lstm, 10)]
for name, model, length in models:
    validation_data.get_y_pred(model, length)
    print('%30s %.9f %.9f' % (name, 
                              metrics.accuracy_score(validation_data.df['y_train'], validation_data.df['y_pred']), 
                              metrics.f1_score(validation_data.df['y_train'], validation_data.df['y_pred'])))

                                  Accuracy    F1 Score
                      Keep All 0.501288660 0.667811159
                     Keep Some 0.458762887 0.486552567
           Logistic Regression 0.528350515 0.652751423
                   Naive Bayes 0.506443299 0.661959400
                 Dense Network 0.523195876 0.613778706
          LSTM with 1 timestep 0.458762887 0.618181818
        LSTM with 10 timesteps 0.492268041 0.360389610


In [59]:
validation_data, _ = data.split(0.5)
for name, model, length in models:
    validation_data.get_y_pred(model, length)
    validation_data.df[name] = validation_data.df['y_pred']
validation_data.show_score(col=['x_train'] + [i[0] for i in models] + ['y_train'])

## Evaluation of lstm with binary vectors

In [155]:
lstm2 = Sequential()
lstm2.add(LSTM(128, input_shape=(None, 128)))
lstm2.add(Dropout(0.2))
lstm2.add(Activation('sigmoid'))
lstm2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', f1])
lstm2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
Total params: 131,584
Trainable params: 131,584
Non-trainable params: 0
_________________________________________________________________


### The result is not satisfying

#### Too inaccurate

#### Too slow

#### May need even more time if all features are used as inputs

In [207]:
data = sd.ScoreData.load('score_data/2.pkl')
testing_data, training_data = data.split()
x_train, y_train = training_data.generate_data_2(length=16)
x_test, y_test = testing_data.generate_data_2(length=16)
logger = NBatchLogger(display=50)
early_stop = EarlyStopping(monitor='f1', mode='max', min_delta=0, patience=2000, verbose=0, restore_best_weights=False)
lstm2.fit(x_train, y_train, epochs=2000, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop, logger], verbose=0)

step: 50/2000 ...  - loss: 0.3465 - acc: 0.9900 - f1: 0.7264 - val_loss: 0.3314 - val_acc: 0.9831 - val_f1: 0.3801
step: 100/2000 ...  - loss: 0.3464 - acc: 0.9902 - f1: 0.7331 - val_loss: 0.3317 - val_acc: 0.9830 - val_f1: 0.3705
step: 150/2000 ...  - loss: 0.3463 - acc: 0.9902 - f1: 0.7348 - val_loss: 0.3318 - val_acc: 0.9827 - val_f1: 0.3617
step: 200/2000 ...  - loss: 0.3460 - acc: 0.9904 - f1: 0.7403 - val_loss: 0.3319 - val_acc: 0.9828 - val_f1: 0.3596
step: 250/2000 ...  - loss: 0.3460 - acc: 0.9903 - f1: 0.7382 - val_loss: 0.3317 - val_acc: 0.9829 - val_f1: 0.3600
step: 300/2000 ...  - loss: 0.3460 - acc: 0.9903 - f1: 0.7372 - val_loss: 0.3319 - val_acc: 0.9828 - val_f1: 0.3569
step: 350/2000 ...  - loss: 0.3459 - acc: 0.9904 - f1: 0.7402 - val_loss: 0.3318 - val_acc: 0.9829 - val_f1: 0.3602
step: 400/2000 ...  - loss: 0.3460 - acc: 0.9904 - f1: 0.7403 - val_loss: 0.3317 - val_acc: 0.9828 - val_f1: 0.3580
step: 450/2000 ...  - loss: 0.3461 - acc: 0.9904 - f1: 0.7405 - val_loss:

#### Size of inputs affect the accuracy?

In [266]:

data = ScoreData.load('score_data/2.pkl')
validation_data, _ = data.split(0.5)
from sklearn import metrics
print('%30s %11s %11s' % ('', 'Accuracy', 'F1 score'))
for l in range(1, 30):
    models = [('LSTM with binary %d' % l, lstm2, l)]
    for name, model, length in models:
        validation_data = validation_data.merge_binary((lstm2.predict(validation_data.generate_data_2(length=length)[0]) > 0.5) * 1, skip_features=True)
        print('%30s %.9f %.9f' % (name, 
                                  metrics.accuracy_score(validation_data.df['y_train'], validation_data.df['y_pred']), 
                                  metrics.f1_score(validation_data.df['y_train'], validation_data.df['y_pred'])))

                                  Accuracy    F1 score
            LSTM with binary 1 0.497542998 0.443537415
            LSTM with binary 2 0.472123369 0.340740741
            LSTM with binary 3 0.465577596 0.273015873
            LSTM with binary 4 0.457044674 0.252365931
            LSTM with binary 5 0.461797753 0.261941448
            LSTM with binary 6 0.473509934 0.251177394
            LSTM with binary 7 0.477546550 0.239234450
            LSTM with binary 8 0.484187568 0.255118110
            LSTM with binary 9 0.480998914 0.255451713
           LSTM with binary 10 0.485342020 0.273006135
           LSTM with binary 11 0.483170467 0.285285285
           LSTM with binary 12 0.491323210 0.303120357
           LSTM with binary 13 0.494577007 0.310650888
           LSTM with binary 14 0.491874323 0.305185185
           LSTM with binary 15 0.489707476 0.306332842
           LSTM with binary 16 0.486486486 0.304538799
           LSTM with binary 17 0.485405405 0.302052786
          

In [264]:
validation_data.show_score()

## Start implementing the postprocessor

In [213]:
lstm2.save('models/LSTM_binary.h5')
from keras.models import load_model
lstm3 = load_model('models/LSTM_binary.h5', custom_objects={'f1': f1})

In [101]:
import keras
class NBatchLogger(keras.callbacks.Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_epoch_end(self, batch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log = ''
            for (k, v) in self.metric_cache.items():
                val = v / self.display
                if abs(val) > 1e-3:
                    metrics_log += ' - %s: %.4f' % (k, val)
                else:
                    metrics_log += ' - %s: %.4e' % (k, val)
            print('step: {}/{} ... {}'.format(self.step,
                                          self.params['epochs'],
                                          metrics_log))
            self.metric_cache.clear()